In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import re

In [4]:
start = time.time()
# Initialize the webdriver
# de manière basique
# driver = webdriver.Chrome()

# avec ChromeDriverManager mieux car update du driver automatiquement
driver = webdriver.Chrome(ChromeDriverManager().install())

# si on a télécharger le driver directement dans le dossier 
# service = Service(executable_path="./chromedriver.exe")
# driver = webdriver.Chrome(service=service)

# Navigate to the website
# driver.get("https://www.billboard.com/charts/hot-100/")
driver.get("https://www.billboard.com/charts/billboard-global-200/")
# driver.get("https://www.billboard.com/charts/billboard-200/")

# Wait for the page to load
# driver.implicitly_wait(10)
time.sleep(5)

# reject all cookies
driver.find_element(By.XPATH,"/html/body/div[5]/div[2]/div/div/div[2]/div/div/button[1]").click()

iterateur100 = list(range(1,110,11))
iterateur100.remove(1)
iterateur200 = list(range(1,220,11))
iterateur200.remove(1)

time.sleep(50)
# nb_max 111 ou 221
def scraper(list,xpath,iterateur,nb_max):
    for i in range(2,nb_max):
        if i not in iterateur : 
            WebElement = driver.find_element(By.XPATH,xpath%(i))
            list.append(WebElement.text)

    return list

XpathTitle = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/h3"
XpathArtist = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/span"
XpathRank = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[1]/span "
XpathLastWeek = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[4]/span"
XpathPeakPosition = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[5]/span"
XpathWeeksOnChart = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[6]/span"

allXpath = [XpathTitle,XpathArtist,XpathRank,XpathLastWeek,XpathPeakPosition,XpathWeeksOnChart]

title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]

allList = [title,artist,rank,last_week,peak_pos,weeks_on_chart]

i = [scraper(i,j,iterateur200,221) for i, j in zip(allList, allXpath)]

# for i, j in zip(allList, allXpath):
#     i = scraper(i,j,iterateur200,221)

df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts'])
print(df)

# Close the webdriver
driver.close()

end = time.time()
temps = end - start
print("temps d'execution :",temps,"s")

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 41.1MB/s]
C:\Users\thoma\AppData\Local\Temp\ipykernel_19768\1624214680.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


                            Title                        Artist Rank  \
0                         Flowers                   Miley Cyrus    1   
1                       Kill Bill                           SZA    2   
2    Bzrp Music Sessions, Vol. 53            Bizarrap & Shakira    3   
3                          Unholy        Sam Smith & Kim Petras    4   
4                       Calm Down           Rema & Selena Gomez    5   
..                            ...                           ...  ...   
195                    In The End                   Linkin Park  196   
196                       Ferrari  James Hype & Miggy Dela Rosa  197   
197          Just The Way You Are                    Bruno Mars  198   
198              Call Out My Name                    The Weeknd  199   
199                     Good Days                           SZA  200   

    Last Week Peak Positon Weeks on charts  
0           1            1               4  
1           3            1               9  


In [5]:
song_artists = list(zip(df['Artist'],df["Title"]))
song_artists

[('Miley Cyrus', 'Flowers'),
 ('SZA', 'Kill Bill'),
 ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'),
 ('Sam Smith & Kim Petras', 'Unholy'),
 ('Rema & Selena Gomez', 'Calm Down'),
 ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"),
 ('Harry Styles', 'As It Was'),
 ('RAYE Featuring 070 Shake', 'Escapism'),
 ('David Guetta & Bebe Rexha', "I'm Good (Blue)"),
 ('Taylor Swift', 'Anti-Hero'),
 ('Morgan Wallen', 'Last Night'),
 ('NewJeans', 'OMG'),
 ('Miguel', 'Sure Thing'),
 ('NewJeans', 'Ditto'),
 ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"),
 ('Yandel & Feid', 'Yandel 150'),
 ('Manuel Turizo', 'La Bachata'),
 ('Stephen Sanchez', 'Until I Found You'),
 ('JVKE', 'Golden Hour'),
 ('The Weeknd', 'Die For You'),
 ('Meghan Trainor', 'Made You Look'),
 ('Arcangel & Bad Bunny', 'La Jumpa'),
 ('Beyonce', 'Cuff It'),
 ('OneRepublic', "I Ain't Worried"),
 ('The Weeknd & Ariana Grande', 'Save Your Tears'),
 ('Tom Odell', 'Another Love'),
 ('Drake & 21 Savage', 'Rich Flex'),
 ('SZA',

In [6]:
# #modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):
    
    if 'X' in song_artists[i][0] and 'Lil Nas X' not in song_artists[i][0]:
        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("X", "&")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Fuerza Regida & Grupo Frontera', 'Bebe Dame')
('Peso Pluma & Natanael Cano', 'PRC')
('Carin Leon & Grupo Frontera', 'Que Vuelvas')
('Junior H & Oscar Maydon', 'Fin de Semana')
('Imagine Dragons & JID', 'Enemy')
('TOMORROW & TOGETHER', 'Sugar Rush Ride')
('Fuerza Regida & Natanael Cano', 'Ch y La Pizza')


In [7]:
for i in range(len(song_artists)):

    if 'vs.' in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("vs.", "")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

print(song_artists)

('Lana Del Rey  Cedric Gervais', 'Summertime Sadness')
[('Miley Cyrus', 'Flowers'), ('SZA', 'Kill Bill'), ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'), ('Sam Smith & Kim Petras', 'Unholy'), ('Rema & Selena Gomez', 'Calm Down'), ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"), ('Harry Styles', 'As It Was'), ('RAYE Featuring 070 Shake', 'Escapism'), ('David Guetta & Bebe Rexha', "I'm Good (Blue)"), ('Taylor Swift', 'Anti-Hero'), ('Morgan Wallen', 'Last Night'), ('NewJeans', 'OMG'), ('Miguel', 'Sure Thing'), ('NewJeans', 'Ditto'), ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"), ('Yandel & Feid', 'Yandel 150'), ('Manuel Turizo', 'La Bachata'), ('Stephen Sanchez', 'Until I Found You'), ('JVKE', 'Golden Hour'), ('The Weeknd', 'Die For You'), ('Meghan Trainor', 'Made You Look'), ('Arcangel & Bad Bunny', 'La Jumpa'), ('Beyonce', 'Cuff It'), ('OneRepublic', "I Ain't Worried"), ('The Weeknd & Ariana Grande', 'Save Your Tears'), ('Tom Odell', 'Another Love'), ('Drake & 21 Sa

In [8]:
#modification du nom de l'artiste pour la recherche Genius

for i in range(len(song_artists)):

    if " With " in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace(" With ", " ")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

print(song_artists)

('Kane Brown Katelyn Brown', 'Thank God')
[('Miley Cyrus', 'Flowers'), ('SZA', 'Kill Bill'), ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'), ('Sam Smith & Kim Petras', 'Unholy'), ('Rema & Selena Gomez', 'Calm Down'), ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"), ('Harry Styles', 'As It Was'), ('RAYE Featuring 070 Shake', 'Escapism'), ('David Guetta & Bebe Rexha', "I'm Good (Blue)"), ('Taylor Swift', 'Anti-Hero'), ('Morgan Wallen', 'Last Night'), ('NewJeans', 'OMG'), ('Miguel', 'Sure Thing'), ('NewJeans', 'Ditto'), ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"), ('Yandel & Feid', 'Yandel 150'), ('Manuel Turizo', 'La Bachata'), ('Stephen Sanchez', 'Until I Found You'), ('JVKE', 'Golden Hour'), ('The Weeknd', 'Die For You'), ('Meghan Trainor', 'Made You Look'), ('Arcangel & Bad Bunny', 'La Jumpa'), ('Beyonce', 'Cuff It'), ('OneRepublic', "I Ain't Worried"), ('The Weeknd & Ariana Grande', 'Save Your Tears'), ('Tom Odell', 'Another Love'), ('Drake & 21 Savage', 'Rich 

In [9]:
#modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):

    if "Coolio Featuring L.V." in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("Coolio Featuring L.V.", "Colio")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Colio', "Gangsta's Paradise")


In [10]:
#modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):

    if "Quevedo & Mike Towers" in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("Quevedo & Mike Towers", "Quevedo")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Quevedo', 'Playa del Ingles')


In [11]:
song_artists.remove(('Carin Leon & Grupo Frontera', 'Que Vuelvas'))
print(('Carin Leon & Grupo Frontera', 'Que Vuelvas') in song_artists)

False


In [47]:
driver_genius = webdriver.Chrome(ChromeDriverManager().install())

    # ouvrir le site et accepter les cookies

driver_genius.get("https://genius.com/")
driver_genius.implicitly_wait(10)
driver_genius.find_element(By.ID,"onetrust-accept-btn-handler").click()
time.sleep(3)
 
song,artist,date,producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags = [],[],[],[],[],[],[],[]


for i in range(59,100):

    driver_genius.get("https://genius.com/")

    # activer la recherche
    element = driver_genius.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/form/input")
    element.send_keys(song_artists[i][1]+' '+song_artists[i][0], Keys.ENTER)
    
    song.append(song_artists[i][1])
    artist.append(song_artists[i][0])

    driver_genius.implicitly_wait(10)

    # clicker sur la chanson
    try :

        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a/div[2]").click()
        driver_genius.implicitly_wait(10)
    
    except:

        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a").click()
        driver_genius.implicitly_wait(10)

    driver_genius.implicitly_wait(10)

    #scroll to credits

    target_credits = driver_genius.find_element(By.CSS_SELECTOR,"div.SongInfo__Title-nekw6x-1")
    driver_genius.execute_script("arguments[0].scrollIntoView();", target_credits)

    driver_genius.implicitly_wait(10)


    # Si le bouton expand est présent

    try: 
        #click on expand if this button exist

        all_elements_after_target_credit = target_credits.find_elements(By.XPATH, ".//following::*")

        # trouve le 1er element dont le tag_name est "button"

        for element in all_elements_after_target_credit:
            if element.tag_name == "button":
                element.click()
                break

        driver_genius.implicitly_wait(10)
    
    except:

        # Le bouton Expand n'a pas été trouvé sur la page
        print("Le bouton n'existe pas sur la page")

    # Localiser la balise "div" contenant la date en utilisant son contenu texte

    date_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Release Date')]")
    driver_genius.implicitly_wait(10)
    # Extraire la date à partir du texte en utilisant des expressions régulières
    if date_element:
        date_text = date_element.find_element(By.XPATH,'./following-sibling::div').text
        date_regex = r"\w+\s+\d{1,2},\s+\d{4}"
        time.sleep(2)
        match = re.search(date_regex, date_text)
        if match:

            date.append(match.group())

        else:
            
            date.append("Nan")

    time.sleep(2)

    #Recherche des producteurs,ecrivains,publisher,distributor et tags

    ##################################################PRODUCTEURS#############################################
    try:
        producteurs_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Produced By')]")
        producteurs_text.append(producteurs_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        producteurs_text.append("Nan")
    ############################################ECRIVAIN#############################################
    driver_genius.implicitly_wait(10)
    try:
        ecrivains_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Written By')]")
        ecrivains_text.append(ecrivains_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:  
        ecrivains_text.append("Nan")
    ########################################PUBLISHER##################################################
    driver_genius.implicitly_wait(10)
    try:
        publisher_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Publisher')]")
        publisher_text.append(publisher_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        publisher_text.append("Nan")
    #######################################DISTIBUTEURS###############################################
    driver_genius.implicitly_wait(10)
    try:
        distributor_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Distributor')]")
        distributor_text.append(distributor_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        distributor_text.append("Nan")

    #############################################TAGS#####################################################
    driver_genius.implicitly_wait(10)

    try:
        tag_pos = driver_genius.find_element(By.CLASS_NAME,"SongTags__Title-xixwg3-0")
        driver_genius.execute_script("arguments[0].scrollIntoView();", tag_pos)
        name = driver_genius.find_elements(By.CLASS_NAME, "SongTags__Container-xixwg3-1")

        driver_genius.implicitly_wait(10)

        tag=[]

        for elt in name:

            tag.append(elt.text)

        tags = ''.join(set(tag))  
        
        Tags.append(tags.replace('\n',', '))
        driver_genius.implicitly_wait(10)

    except:

        Tags.append("Nan")
        driver_genius.implicitly_wait(10)
    # on revient à l'accueil
    
        driver_genius.get("https://genius.com/")
    # verif = (date[i],song[i])
    print(date[i-59],song[i-59])
    print(i)
    driver_genius.implicitly_wait(10)
    
print(date, producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags)
# df_genius = pd.DataFrame( {'Realise Date': date, 'Producers': producteurs_text, 'Writes' : ecrivains_text ,'Publisher':publisher_text,'Distributors':distributor_text, 'Tags':distributor_text})

C:\Users\thoma\AppData\Local\Temp\ipykernel_19768\2732567698.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_genius = webdriver.Chrome(ChromeDriverManager().install())


November 12, 2004 Mockingbird
59
October 21, 2022 Lavender Haze
60
March 28, 2012 Sweater Weather
61
July 23, 2022 Hype Boy
62
January 13, 2023 Fin de Semana
63
September 10, 2021 Shivers
64
December 9, 2022 Leao
65
August 19, 2022 Feliz Cumpleanos Ferxxo
66
December 15, 2022 Let Go
67
Nan People
68
October 15, 2021 Easy On Me
69
December 22, 2022 Gato de Noche
70
February 10, 2020 Shut Up My Moms Calling
71
December 3, 2012 When I Was Your Man
72
April 14, 2022 About Damn Time
73
May 6, 2022 Ojitos Lindos
74
December 9, 2022 Low
75


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.SongInfo__Title-nekw6x-1"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00C56643]
	(No symbol) [0x00BEBE21]
	(No symbol) [0x00AEDA9D]
	(No symbol) [0x00B21342]
	(No symbol) [0x00B2147B]
	(No symbol) [0x00B58DC2]
	(No symbol) [0x00B3FDC4]
	(No symbol) [0x00B56B09]
	(No symbol) [0x00B3FB76]
	(No symbol) [0x00B149C1]
	(No symbol) [0x00B15E5D]
	GetHandleVerifier [0x00ECA142+2497106]
	GetHandleVerifier [0x00EF85D3+2686691]
	GetHandleVerifier [0x00EFBB9C+2700460]
	GetHandleVerifier [0x00D03B10+635936]
	(No symbol) [0x00BF4A1F]
	(No symbol) [0x00BFA418]
	(No symbol) [0x00BFA505]
	(No symbol) [0x00C0508B]
	BaseThreadInitThunk [0x76147D69+25]
	RtlInitializeExceptionChain [0x7728BB9B+107]
	RtlClearBits [0x7728BB1F+191]


In [41]:
print(len(date))
print(len(producteurs_text))
print(len(ecrivains_text))
print(len(publisher_text))
print(len(distributor_text))
print(len(Tags))

6
6
6
6
6
6


In [42]:
print(date[90])

IndexError: list index out of range

In [43]:
for i in range(len(date)):

    if date[i]=="Nan":

        print(i)

In [44]:
print(song_artists[42])

('Zach Bryan', 'Something In The Orange')


In [45]:
df_genius_100 = pd.DataFrame( { 'song':song, 'artist':artist,'date':date, 'Producers': producteurs_text, 'Writes' : ecrivains_text ,'Publisher':publisher_text,'Distributors':distributor_text,'Tags':Tags})
df_genius_100

,song,artist,date,Producers,Writes,Publisher,Distributors,Tags
0,X Si Volvemos,Karol G x Romeo Santos,"February 2, 2023",Ovy On The Drums,"KAROL G, Ovy On The Drums & Romeo Santos",Nan,Nan,"Pop, Reggaetón, Colombia, En Español, Latin Po..."
1,Hey Mor,Ozuna Featuring Feid,"October 7, 2022",Sky Rompiendo,"Ozuna, Feid, Sencillo305, Rios (PR), Sky Rompi...","Ozuna Worldwide, Kobalt Music, Warner/Chappell...",Sony Music Latin & The Orchard,"Pop, Reggaetón, Colombia, Puerto Rico, En Espa..."
2,Under The Influence,Chris Brown,"October 4, 2019",Kiddominant,"Tiffany McKie, Davido, Kiddominant & Chris Brown","Copyright Control, Universal Music Group, Sony...",Nan,"R&B, Alternative R&B, Alternative, DMV"
3,Just Wanna Rock,Lil Uzi Vert,"October 17, 2022",MCVertt & Synthetic,"Lil Uzi Vert, MCVertt & Synthetic","Create Music Group, NirvanaDigitalPublishing, ...",Warner Music Group,"Rap, Dance-Pop, Dance, Electronic Trap, Electr..."
4,Players,Coi Leray,"November 30, 2022",Johnny Goldstein,"Coi Leray, Johnny Goldstein, Worldwide Fresh &...",Nan,Universal Music Group,"Rap, New Wave, Nu Disco"
5,"Bzrp Music Sessions, Vol. 52",Bizarrap & Quevedo,"July 6, 2022",Bizarrap,"Santiago Alvarado, Quevedo & Bizarrap",Nan,The Orchard,"Pop, Rap, Pop Rap, Trap, Spanish Rap, House, S..."
